In [2]:
import librosa
import sounddevice as sd
from pystoi.stoi import stoi
from librosa.core import stft, istft
import os
import h5py
from natsort import natsorted
import numpy as np
import math as m
from sklearn import preprocessing

### specify paths

In [53]:
orig_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/test_data_other'
raw_folder = 'test_16000_all_other'

### extract log and phase:
test_log, test_phase: log and phase of test files with zero elements in between to construct the context window.( for two talker data_not clean)
phase does not have zero elements.

In [64]:
log_folder = 'test_log'
phase_folder = 'test_phase'
a = os.listdir(os.path.join(orig_path,raw_folder))
a = natsorted(a)
if not os.path.exists(os.path.normpath(os.path.join(write_path,log_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,log_folder)))
if not os.path.exists(os.path.normpath(os.path.join(write_path,phase_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,phase_folder)))
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.normpath(os.path.join(orig_path,raw_folder,filename)))
        b = natsorted(b)
        if not os.path.exists(os.path.normpath(os.path.join(write_path,log_folder,filename))):
            os.mkdir(os.path.normpath(os.path.join(write_path,log_folder,filename)))
        if not os.path.exists(os.path.join(write_path,phase_folder,filename)):
            os.mkdir(os.path.normpath(os.path.join(write_path,phase_folder,filename)))
        for filename2 in b:
            wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder)+"/"+filename+'/'+filename2, sr=16000)
            max_wav=np.max(wav_data)
            wav_data = wav_data/max_wav
            framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
            abslt=np.absolute(framed_data)**2
            dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
            for i in range(m.floor(w/2)):
                dft_signal=np.insert(dft_signal,0,0,axis = 1)
            data_phase=np.angle(framed_data)   
            np.savetxt(os.path.join(write_path,log_folder,str(filename),str(filename2.replace('.wav','.txt'))), dft_signal.T, delimiter=',')
            np.savetxt(os.path.join(write_path,phase_folder,str(filename),str(filename2.replace('.wav','.txt'))), data_phase.T, delimiter=',')


### extract log and phase:
extract log and phases of data without the zero element in between.

In [65]:
log_folder = 'test_log2'
phase_folder = 'test_phase2'
if not os.path.exists(os.path.normpath(os.path.join(write_path,log_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,log_folder)))
if not os.path.exists(os.path.normpath(os.path.join(write_path,phase_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,phase_folder)))
log_path = os.path.normpath(os.path.join(write_path,log_folder))
f = h5py.File(log_path+'.hdf5', 'w')
d = f.create_dataset(log_folder, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
a = os.listdir(os.path.normpath(os.path.join(orig_path,raw_folder)))
a = natsorted(a)
wav = []
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.join(orig_path,raw_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,log_folder,filename)):
            os.mkdir(os.path.join(write_path,log_folder,filename))
        if not os.path.exists(os.path.join(write_path,phase_folder ,filename)):
            os.mkdir(os.path.join(write_path, phase_folder, filename))
        for filename2 in b:
            wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder,filename,filename2), sr=16000) 
            max_wav=np.max(wav_data)
            wav_data = wav_data/max_wav
            framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
            abslt=np.absolute(framed_data)**2
            dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
            wav.append(dft_signal.T)
            data_phase=np.angle(framed_data)    
            np.savetxt(os.path.join(write_path,log_folder,filename,str(filename2.replace('.wav','.txt'))), dft_signal.T, delimiter=',')
            np.savetxt(os.path.join(write_path,phase_folder ,filename,str(filename2.replace('.wav','.txt'))), data_phase.T, delimiter=',')
            d.resize(d.shape[0]+len(dft_signal.T), axis=0)   
            d[-1*len(dft_signal.T):] = dft_signal.T
f.close()


### find mean and std of log non context test data.

In [66]:
wav = np.concatenate( wav, axis=0 )
tot_len=len(wav)
dim = wav.shape[1]
lst=0
lst=lst+np.sum(wav,axis=0)
tot_mean=lst/tot_len
lst=0
lst=lst+np.sum((wav - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
np.savetxt(os.path.normpath(os.path.join(write_path,'test_mean_log2.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'test_std_log2.txt')),tot_std)

### apply mean and variance of test data to unnormed test data.

In [67]:
w=1
log_folder = 'test_log2_norm'
a = os.listdir(os.path.normpath(os.path.join(orig_path,raw_folder)))
a = natsorted(a)
if not os.path.exists(os.path.normpath(os.path.join(write_path,log_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,log_folder)))
wav = []
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.join(orig_path,raw_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,log_folder,filename)):
            os.mkdir(os.path.join(write_path,log_folder,filename))
        if not os.path.exists(os.path.join(write_path,phase_folder ,filename)):
            os.mkdir(os.path.join(write_path, phase_folder, filename))
        for filename2 in b:
            wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder,filename,filename2), sr=16000) 
            max_wav=np.max(wav_data)
            wav_data = wav_data/max_wav
            framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
            abslt=np.absolute(framed_data)**2
            dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
            normalized = (dft_signal.T - tot_mean)/tot_std
            np.savetxt(os.path.join(write_path,log_folder,filename,str(filename2.replace('.wav','.txt'))), normalized, delimiter=',')

### apply mean and variance of  train data to test data. has not been run.

In [ ]:
w=1
tot_mean=np.loadtxt(os.path.normpath(os.path.join(train_path,'mean_mixed_log.txt')))
tot_std=np.loadtxt(os.path.normpath(os.path.join(train_path,'std_mixed_log.txt')))
log_folder = 'test_log2_norm2'
a = os.listdir(os.path.normpath(os.path.join(orig_path,raw_folder)))
a = natsorted(a)
if not os.path.exists(os.path.normpath(os.path.join(write_path,log_folder))):
    os.mkdir(os.path.normpath(os.path.join(write_path,log_folder)))
wav = []
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.join(orig_path,raw_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,log_folder,filename)):
            os.mkdir(os.path.join(write_path,log_folder,filename))
        for filename2 in b:
            wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder,filename,filename2), sr=16000) 
            max_wav=np.max(wav_data)
            wav_data = wav_data/max_wav
            framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
            abslt=np.absolute(framed_data)**2
            dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
            normalized = (dft_signal.T - tot_mean)/tot_std
            np.savetxt(os.path.join(write_path,log_folder,filename,str(filename2.replace('.wav','.txt'))), normalized, delimiter=',')
f.close()

### do the same normalization process ans save it in h5 file.(not been tested)

In [60]:
import h5py
import numpy as np
refrmd_file='test_log2'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
h5f = h5py.File(refrmd_file_path+'.hdf5', 'r')
d=h5f[refrmd_file]
tot_len=d.shape[0]
dim = d.shape[1]
lst=0
mixed = h5f[refrmd_file][:]
lst=lst+np.sum(mixed,axis=0)
tot_mean=lst/tot_len
lst=0
mixed = h5f[refrmd_file][:]
lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()
normalized_file = 'test_log2_norm'
normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
f = h5py.File(normalized_file_path+'.hdf5', 'w')
dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
h5f = h5py.File(refrmd_file_path+'.hdf5','r')

mixed = h5f[refrmd_file][:]
scaled=(mixed-tot_mean)/tot_std
dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
dd[-1*scaled.shape[0]:] = scaled

np.savetxt(os.path.normpath(os.path.join(write_path,'test_mean_log.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'test_std_log.txt')),tot_std)
h5f.close()
f.close()


### get the data for clean file.

In [68]:
log_folder = 'test_log'
phase_folder = 'test_phase'
a = os.listdir(os.path.join(orig_path,raw_folder))
a = natsorted(a)
for filename in a:
    if filename=='clean':
        b = os.listdir(os.path.join(orig_path,raw_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,log_folder,filename)):
            os.mkdir(os.path.join(write_path,log_folder,filename))
        for filename2 in b:
            wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder,filename,filename2), sr=16000)  
            max_wav=np.max(wav_data)
            wav_data = wav_data/max_wav
            np.savetxt(os.path.join(write_path,log_folder,str(filename),str(filename2.replace('.wav','.txt'))), wav_data, delimiter=',')

In [42]:

clean_filename = 'clean_data_semi'
clean_path = os.path.normpath(os.path.join(write_path,clean_filename))
a = os.listdir(os.path.normpath(os.path.join(orig_path,raw_folder)))
a = natsorted(a)
clean=[]
f = h5py.File(clean_path+'.hdf5', 'w')
d = f.create_dataset(clean_filename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
for filename in a:
    if filename=='clean':
        b = os.listdir(os.path.join(orig_path,raw_folder,filename))
        b = natsorted(b)
        for i in range(len(a)-1):
            for filename2 in b:
                wav_data, sr = librosa.load(os.path.join(orig_path,raw_folder,filename,filename2), sr=16000)   
                max_wav=np.max(wav_data)
                wav_data = wav_data/max_wav
                framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
                abslt=np.absolute(framed_data)**2
                dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
#                 clean.append(dft_signal.T)
                d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
                d[-1*dft_signal.shape[1]:] = dft_signal.T
f.close()
# x=1
# hf = h5py.File('clean_data.h5', 'w')
# hf.create_dataset('clean_data', data=x)
# hf.close()

In [4]:
import math as m
import numpy as np
import h5py
w = 3
i = 1
refrmd_file = 'ftr_refrmd_test'
log_folder = 'test_log'
phase_folder = 'test_phase'
refrmd_path = os.path.normpath(os.path.join(write_path,refrmd_file))
if not os.path.exists(os.path.normpath(os.path.join(write_path,refrmd_file))):
    os.mkdir(os.path.normpath(os.path.join(write_path,refrmd_file)))
# tot_mean=np.loadtxt('mean.txt')
# tot_std=np.loadtxt('std.txt')
a = os.listdir(os.path.join(write_path,log_folder))
a = natsorted(a)
# f.close()
f = h5py.File(refrmd_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
ftr_refrmd=[]
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.join(write_path,log_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,refrmd_file,filename)):
            os.mkdir(os.path.join(write_path,refrmd_file,filename))
        for filename2 in b:
            ftr=np.loadtxt(os.path.join(write_path,log_folder,filename,filename2),delimiter=',')
#             ftr_refrmd=[]
            for i in range(m.floor(w/2)):
                p=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    temp.extend(ftr[i+p+j])
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
            for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
                k=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    temp.extend(ftr[i+k])
                    k=k+1
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
                if i%1000==0:
                    print(i)
            for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
                p=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
                        temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
                    else:
                        temp.extend(ftr[i+p+j])
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
            print(len(ftr_refrmd[0]))
#             ftr_refrmd=(ftr_refrmd-tot_mean)/tot_std
#             np.savetxt(os.path.normpath(os.path.join(write_path,refrmd_file,str(filename),str(filename2.replace('.wav','.txt')))), ftr_refrmd, delimiter=',')
            
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
f.close()

771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771


In [5]:
import h5py
import numpy as np
refrmd_file='ftr_refrmd_test'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
h5f = h5py.File(refrmd_file_path+'.hdf5', 'r')
d=h5f[refrmd_file]
tot_len=d.shape[0]
dim = d.shape[1]
lst=0
tot_num=(tot_len//10000)*10000
for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[refrmd_file][i:i+10000]
    lst=lst+np.sum(mixed,axis=0)
mixed = h5f[refrmd_file][tot_num:]
lst=lst+np.sum(mixed,axis=0)
print(mixed.shape)
tot_mean=lst/tot_len
lst=0
h5f = h5py.File(refrmd_file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    mixed = h5f[refrmd_file][i:i+10000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
del mixed  
mixed = h5f[refrmd_file][tot_num:]
lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()
normalized_file = 'ftr_refrmd_test_norm'
normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
f = h5py.File(normalized_file_path+'.hdf5', 'w')
dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
h5f = h5py.File(refrmd_file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[refrmd_file][i:i+10000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled
del mixed    
mixed = h5f[refrmd_file][tot_num:]
print(mixed.shape)
scaled=(mixed-tot_mean)/tot_std
dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
dd[-1*scaled.shape[0]:] = scaled

np.savetxt(os.path.normpath(os.path.join(write_path,'test_mean.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'test_std.txt')),tot_std)
h5f.close()
###################################

###################################
# import math as m
# import numpy as np
# import h5py
# w = 3
# i = 1
# refrmd_folder = 'ftr_refrmd_test_norm_semi'
# log_folder = 'test_log_semi'
# phase_folder = 'test_phase_semi'
# refrmd_path = os.path.normpath(os.path.join(write_path,refrmd_folder))
# if not os.path.exists(os.path.normpath(os.path.join(write_path,refrmd_folder))):
#     os.mkdir(os.path.normpath(os.path.join(write_path,refrmd_folder)))
# tot_mean=np.loadtxt(os.path.normpath(os.path.join(write_path,'test_mean_semi.txt')))
# tot_std=np.loadtxt(os.path.normpath(os.path.join(write_path,'test_std_semi.txt')))
# a = os.listdir(os.path.join(write_path,log_folder))
# a = natsorted(a)
# ftr_refrmd=[]
# for filename in a:
#     if filename!='clean':
#         b = os.listdir(os.path.join(write_path,log_folder,filename))
#         b = natsorted(b)
#         if not os.path.exists(os.path.join(write_path,refrmd_folder,filename)):
#             os.mkdir(os.path.join(write_path,refrmd_folder,filename))
#         for filename2 in b:
#             ftr=np.loadtxt(os.path.join(write_path,log_folder,filename,filename2),delimiter=',')
#             ftr_refrmd=[]
#             for i in range(m.floor(w/2)):
#                 p=-m.floor(w/2)
#                 temp = []
#                 for j in range(w):
#                     if np.all(ftr[i]==0):
#                         break
#                     temp.extend(ftr[i+p+j])
#                 if np.any(temp!=[]):
#                     ftr_refrmd.append(temp)
#             for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
#                 k=-m.floor(w/2)
#                 temp = []
#                 for j in range(w):
#                     if np.all(ftr[i]==0):
#                         break
#                     temp.extend(ftr[i+k])
#                     k=k+1
#                 if np.any(temp!=[]):
#                     ftr_refrmd.append(temp)
#                 if i%1000==0:
#                     print(i)
#             for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
#                 p=-m.floor(w/2)
#                 temp = []
#                 for j in range(w):
#                     if np.all(ftr[i]==0):
#                         break
#                     if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
#                         temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
#                     else:
#                         temp.extend(ftr[i+p+j])
#                 if np.any(temp!=[]):
#                     ftr_refrmd.append(temp)
#             print(len(ftr_refrmd[0]))
#             ftr_refrmd=(ftr_refrmd-tot_mean)/tot_std
#             np.savetxt(os.path.normpath(os.path.join(write_path,refrmd_folder,str(filename),str(filename2.replace('.wav','.txt')))), ftr_refrmd, delimiter=',')
# f.close()           

0
(3692, 771)
0
(3692, 771)


In [18]:
import math as m
import numpy as np
import h5py
w = 3
i = 1
refrmd_file = 'ftr_refrmd_test_norm2_semi'
log_folder = 'test_log_semi'
phase_folder = 'test_phase_semi'
refrmd_path = os.path.normpath(os.path.join(write_path,refrmd_file))
if not os.path.exists(os.path.normpath(os.path.join(write_path,refrmd_file))):
    os.mkdir(os.path.normpath(os.path.join(write_path,refrmd_file)))
# tot_mean=np.loadtxt('mean.txt')
# tot_std=np.loadtxt('std.txt')
a = os.listdir(os.path.join(write_path,log_folder))
a = natsorted(a)
# f.close()
f = h5py.File(refrmd_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
ftr_refrmd=[]
ftr_refrmd_norm=[]
for filename in a:
    if filename!='clean':
        b = os.listdir(os.path.join(write_path,log_folder,filename))
        b = natsorted(b)
        if not os.path.exists(os.path.join(write_path,refrmd_file,filename)):
            os.mkdir(os.path.join(write_path,refrmd_file,filename))
        for filename2 in b:
            ftr=np.loadtxt(os.path.join(write_path,log_folder,filename,filename2),delimiter=',')
#             ftr_refrmd=[]
#             ftr_refrmd_norm=[]
            for i in range(m.floor(w/2)):
                p=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    temp.extend(ftr[i+p+j])
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
            for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
                k=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    temp.extend(ftr[i+k])
                    k=k+1
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
                if i%1000==0:
                    print(i)
            for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
                p=-m.floor(w/2)
                temp = []
                for j in range(w):
                    if np.all(ftr[i]==0):
                        break
                    if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
                        temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
                    else:
                        temp.extend(ftr[i+p+j])
                if np.any(temp!=[]):
                    ftr_refrmd.append(temp)
            print(len(ftr_refrmd[0]))
#             ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)
#             np.savetxt(os.path.normpath(os.path.join(write_path,refrmd_file,str(filename),str(filename2.replace('.wav','.txt')))), ftr_refrmd_norm, delimiter=',')
ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)            
d.resize(d.shape[0]+len(ftr_refrmd_norm), axis=0)   
d[-1*len(ftr_refrmd_norm):] = ftr_refrmd_norm
ftr_refrmd=[]
ftr_refrmd_norm=[]
f.close()

771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771


In [45]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 100000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/test_data'
input_name = 'ftr_refrmd_test_norm_semi'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'clean_data_semi'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
tfrecord_name = 'test_wav_norm_semi'
if not os.path.exists(os.path.normpath(os.path.join(Data_path,'tfrecord_files',tfrecord_name))):
    os.mkdir(os.path.normpath(os.path.join(Data_path,'tfrecord_files',tfrecord_name)))
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,'tfrecord_files',tfrecord_name,'filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)


Serializing 12966 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\test_wav_norm_semi\filenum0.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\test_wav_norm_semi\filenum0.tfrecords done!


In [33]:
import h5py
import os
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/test_data'
# write_path = os.path.normpath(os.path.join(write_path,refrmd_file))
h5f = h5py.File(os.path.normpath(os.path.join(write_path,'ftr_refrmd_test_norm_semi.hdf5')),'r')
x = h5f['ftr_refrmd_test_norm_semi']
# h5f.close()
l=x.shape
h5f.close()


In [35]:
import h5py
import os
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/test_data'
# write_path = os.path.normpath(os.path.join(write_path,refrmd_file))
h5f = h5py.File(os.path.normpath(os.path.join(write_path,'clean_data_semi.hdf5')),'r')
x = h5f['clean_data_semi']
# h5f.close()
ll=x.shape
h5f.close()